In [7]:
#### IMPORT LIBRARIES

import numpy as np
# import all skimage related stuff
import skimage.io
from skimage.measure import label
from skimage import filters, exposure

# import all napari related stuff
import napari
from napari.types import ImageData, LabelsData
from magicgui import magicgui

%gui qt
import os

#### PROCESSING FUNCTIONS

def adaptive_local_threshold(image, block_size):
    return filters.threshold_local(image, block_size)

def gamma_level(image, gamma):
    return exposure.adjust_gamma(image, gamma)

def global_threshold_method(image, Threshold_Method):
    if Threshold_Method == 'Isodata':
        thresh = filters.threshold_isodata(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Li':
        thresh = filters.threshold_li(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Mean':
        thresh = filters.threshold_mean(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Minimum':
        thresh = filters.threshold_minimum(image)
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Otsu':
        thresh = filters.threshold_otsu(image)
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Yen':
        thresh = filters.threshold_yen(image)
        tmp_img = image > thresh
        return tmp_img
    if Threshold_Method == 'Triangle':
        thresh = filters.threshold_triangle(image)
        tmp_img = image > thresh
        return tmp_img

#### MAIN WIDGET/PROGRAM HERE

@magicgui(call_button = 'Run Segmentation',
          gamma = {"widget_type": "FloatSlider", 'max': 2},
          block_size = {"widget_type": "SpinBox", 'label': 'Block Size:', 'min': 1, 'max': 20},
          threshold_method = {"choices": ['Isodata', 'Li', 'Mean', 'Minimum', 'Otsu', 'Triangle', 'Yen']},
          layout = 'vertical'
 )
def threshold_widget(image: ImageData, 
                     gamma = 1,
                     block_size = 3,
                     threshold_method = 'Yen',
                     ) -> LabelsData:
    if image is not None:
        # adjust the gamma level
        label_img = gamma_level(image, gamma)
        
        # go through the stack and perform the local threshold
        
        for curr_stack in range(np.shape(label_img)[0]):
            label_img[curr_stack] = adaptive_local_threshold(label_img[curr_stack], block_size)
        
        # finally do a global threshold to calculate optimized value to make it black/white
        label_img = global_threshold_method(label_img, threshold_method)
#         label_img = label(label_img)
#         labels_layer = viewer.add_labels(label_img, name = 'segmentation')

        return label_img

# laod in neuron images via path 

neuron_dir = "/mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/TRPC6 Spacetraining Data/TRPC6 GSK Spacetraining/190131/N1A5/" # change this directory
neuron_file = "000_D_190130_A_N1A5.tif" # change file name

neuron_image = skimage.io.imread(os.path.join(neuron_dir,neuron_file))

viewer = napari.Viewer()
viewer.add_image(neuron_image, name = 'Neuron')
viewer.window.add_dock_widget(threshold_widget, area = 'right')
# threshold_widget()